In [91]:
import numpy as np

class KNearestNeighbors():
    def __init__(self, X, y, n_neighbors=5, weights = 'uniform', p=2, metric = 'minkowski'):
        self.X = X
        self.y = y
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p
        self.metric = metric
        
    def distance(self, x, y):
      #  if self.p==2 & self.weights =='uniform':
            
        return np.sqrt(np.sum((x-y)**2, axis=1))
    
    def voting():
        pass
    
    
    
    def kneighbors(self, X_test, return_distance=False):
        neigh_ind = np.array([np.argsort(self.distance(x_test, X))[:self.n_neighbors] for x_test in X_test])
        dist = np.array([np.sort(self.distance(x_test, X))[:self.n_neighbors] for x_test in X_test])
        
        if return_distance:
            return dist, neigh_ind
        
        return neigh_ind
    
    def predict(self, X_test):
        neighbors = self.kneighbors(X_test)
        y_pred = np.array([np.argmax(np.bincount(y[neighbor])) for neighbor in neighbors])
        
        return y_pred
    
    def predict_proba(self,X):
        pass
    
    def score(self, X, y):
        pass


In [92]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples = 1000, n_features=2, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=3, random_state=21)


mu = np.mean(X, 0)
sigma = np.std(X, 0)



In [106]:
knn = KNearestNeighbors(X, y)

In [153]:
n = knn.predict([[1,2],[2,3], [-0.7,-1.1]])

In [154]:
n.shape

(3,)

In [155]:
n

array([0, 0, 2])

In [60]:
X[n]

array([[[0.89136445, 1.92837291],
        [0.87605038, 1.88236345],
        [0.99383688, 1.8128406 ],
        [0.82639348, 2.07056827],
        [0.77210681, 2.06059441]],

       [[1.48212723, 2.35517433],
        [1.62766539, 2.2225935 ],
        [1.51318685, 1.79572479],
        [2.80391185, 1.96383103],
        [1.21224695, 1.75865329]]])

In [149]:
X,y

(array([[ 1.74929402,  0.68040021],
        [-0.71052614,  0.72434899],
        [-0.78379194, -1.11312347],
        ...,
        [-2.39382569,  2.15217557],
        [ 1.62359469, -0.3126842 ],
        [-0.16292017,  1.20081947]]),
 array([0, 1, 2, 0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 1, 2, 1, 2, 0, 0, 2,
        2, 0, 2, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 2, 1, 1, 2,
        0, 0, 2, 0, 0, 1, 2, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 2, 2, 2, 2,
        1, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 1, 1, 0, 0, 1, 1, 0, 2, 2, 1, 0,
        1, 2, 0, 0, 2, 1, 2, 0, 2, 1, 0, 1, 2, 0, 2, 2, 1, 1, 2, 2, 0, 2,
        2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 0, 1, 1,
        2, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 1, 2, 2,
        1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 1, 0, 2, 2, 0, 2, 2, 0, 0,
        2, 2, 0, 2, 0, 0, 1, 2, 1, 1, 1, 0, 0, 2, 1, 2, 1, 2, 2, 0, 1, 2,
        0, 1, 1, 0, 1, 1, 0, 1, 2, 2, 1, 2, 1, 2, 0, 1, 2, 1, 1, 0, 2, 0,
        0, 2, 2, 1, 2, 1, 0, 